In [109]:
import polars as pl
df = pl.read_csv('../datasets/competencia_01_crudo.csv',ignore_errors=True)
df = df.join(df['numero_de_cliente'].value_counts(),on='numero_de_cliente')
df = df.with_columns(
    pl.when(df['counts'] == 1).then('BAJA+1')\
      .when(df['counts'] == 2).then('BAJA+2')\
      .when(df['counts'] == 3).then('CONTINUA')\
).rename({'literal':'clase_ternaria'}).drop(columns='counts')

In [110]:
df.filter(pl.col('foto_mes')==202103)['clase_ternaria'].value_counts()

clase_ternaria,counts
str,u32
"""CONTINUA""",161311
"""BAJA+2""",964
"""BAJA+1""",1049


Identificamos un caso raro

In [111]:
df.pivot(columns='foto_mes',index='numero_de_cliente',
         values='numero_de_cliente',aggregate_function='count').fill_null(0)\
            .filter((pl.col('202103')==1)&(pl.col('202105')==1)&(pl.col('202104')==0))

numero_de_cliente,202103,202104,202105
i64,u32,u32,u32
95380426,1,0,1


In [112]:
mask = df['numero_de_cliente'] == 95380426
df = df.with_columns(pl.when(mask).then('continua').otherwise(df['clase_ternaria']).alias('clase_ternaria'))


In [113]:
df.filter(pl.col('foto_mes')==202103)['clase_ternaria'].value_counts()

clase_ternaria,counts
str,u32
"""CONTINUA""",161311
"""continua""",1
"""BAJA+2""",963
"""BAJA+1""",1049


In [114]:
import pathlib
path: pathlib.Path = "../datasets/competencia_01.csv"

In [115]:
df.write_csv(path,separator=',')